v2: denAI

LOAD AND PREPROCESS DATA

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score

import xgboost as xgb
import hmmlearn.hmm as hmm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import warnings
warnings.filterwarnings("ignore")

# Load Data
df = pd.read_csv("C:\\Users\\User\\Documents\\AiTradingV1\\Binance_BTCUSDT_1h (1).csv")

# Data Cleaning: Remove rows with zero volume
df = df[df['Volume BTC'] != 0]
df = df[df['Volume USDT'] != 0]
df.drop(columns=['tradecount'], inplace=True, errors='ignore')

# Normalize column names
df.columns = df.columns.str.lower().str.strip()
df.reset_index(drop=True, inplace=True)

# Display dataset info
print(df.info())
df.tail()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53984 entries, 0 to 53983
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         53984 non-null  object 
 1   symbol       53984 non-null  object 
 2   open         53984 non-null  float64
 3   high         53984 non-null  float64
 4   low          53984 non-null  float64
 5   close        53984 non-null  float64
 6   volume btc   53984 non-null  float64
 7   volume usdt  53984 non-null  float64
dtypes: float64(6), object(2)
memory usage: 3.3+ MB
None


,date,symbol,open,high,low,close,volume btc,volume usdt
53979,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4239.503586
53980,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,19241.058300
53981,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,31282.312670
53982,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,100304.823567
53983,2017-08-17 04:00:00,BTCUSDT,4261.48,4313.62,4261.32,4308.83,47.181009,202366.138393


INDICATORS

In [58]:
def add_indicators(df):
    # Ensure 'close' and 'open' columns exist
    if 'close' not in df.columns or 'open' not in df.columns:
        raise ValueError("DataFrame must contain 'close' and 'open' columns.")
    
    df['volume'] = df['volume btc']

    # Faster RSI (7-period instead of 14)
    delta = df['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).ewm(span=7, adjust=False).mean()
    avg_loss = pd.Series(loss).ewm(span=7, adjust=False).mean()
    rs = avg_gain / (avg_loss + 1e-9)  # Avoid division by zero
    df['rsi'] = 100 - (100 / (1 + rs))

    # Faster MACD (6,13,5)
    short_ema = df['close'].ewm(span=6, adjust=False).mean()
    long_ema = df['close'].ewm(span=13, adjust=False).mean()
    df['macd'] = short_ema - long_ema
    df['macd_signal'] = df['macd'].ewm(span=5, adjust=False).mean()

    # Shorter EMAs (20 & 50) for trend confirmation
    df['ema_20'] = df['close'].ewm(span=20, adjust=False).mean()
    df['ema_50'] = df['close'].ewm(span=50, adjust=False).mean()

    # ATR (5) for faster volatility detection
    df['tr'] = np.maximum.reduce([
        df['high'] - df['low'],
        np.abs(df['high'] - df['close'].shift(1)),
        np.abs(df['low'] - df['close'].shift(1))
    ])
    df['atr'] = df['tr'].rolling(window=5).mean()

    # VWAP (Volume Weighted Average Price)
    df['vwap'] = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum() / df['volume'].cumsum()

    # ADX (Trend Strength)
    df['adx'] = df['atr'].rolling(window=14).mean()

    # **Gap Calculation**
    df['gap'] = df['open'] - df['close'].shift(1)
    df['gap_percent'] = df['gap'] / df['close'].shift(1) * 100

    return df

df = add_indicators(df)

def add_trade_signal(df):
    # Ensure all required columns exist
    required_cols = ['macd', 'macd_signal', 'rsi', 'adx', 'atr', 'vwap', 'close', 'volume', 'gap']
    for col in required_cols:
        if col not in df.columns:
            df[col] = np.nan  # Placeholder if missing

    # Adaptive Position Sizing (Higher Volatility -> Smaller Size)
    df['position_size'] = 1.0 / np.clip(df['atr'], 0.01, 2.0)
    df['position_size'] = np.clip(df['position_size'], 0.5, 1.5)

    # Define Trade Signals for Intraday Trading + Gap Trading
    df['trade_signal'] = np.where(
        (
            (df['macd'] > df['macd_signal']) &  # MACD bullish crossover
            (df['rsi'] < 60) &  # RSI is not overbought
            (df['adx'] >  20) &  # Trend Strength Filter 
            (df['ema_20'].diff() > 0) & (df['ema_50'].diff() > 0) &  # EMAs trending up
            ((df['close'] > df['vwap'])) &  # Allow breakout OR VWAP
            (df['volume'] > df['volume'].rolling(10).mean())
        ) |
        (
            (df['gap_percent'] > 1.5) &  # Bullish Gap > 1.5%
            (df['volume'] > df['volume'].rolling(10).mean())  # Volume confirmation
        ),
        1,  # Buy

        np.where(
            (
                (df['macd'] < df['macd_signal']) &  # MACD bearish crossover
                (df['rsi'] > 40) &  # RSI is not oversold
                (df['adx'] > 20) &
                (df['ema_20'].diff() < 0) & (df['ema_50'].diff() < 0) &  # EMAs trending down
                ((df['close'] < df['vwap'])) &
                (df['volume'] > df['volume'].rolling(10).mean())
            ) |
            (
                (df['gap_percent'] < -1.5) &  # Bearish Gap < -1.5%
                (df['volume'] > df['volume'].rolling(10).mean())  # Volume confirmation
            ),
            2,  # Sell

            0  # Hold
        )
    )

    df['take_profit'] = df['close'] + (df['atr'] * 2.8) 
    df['stop_loss'] = np.where(df['trade_signal'] == 1, df['close'] - (df['atr'] * 2.2),
                        np.where(df['trade_signal'] == 2, df['close'] + (df['atr'] * 2.2), np.nan))

    return df

df = add_trade_signal(df)

# Check if signals are properly assigned
print(df['trade_signal'].value_counts())


trade_signal
0    49828
2     2224
1     1932
Name: count, dtype: int64


In [59]:
print(df.tail())

                      date   symbol     open     high      low    close  \
53979  2017-08-17 08:00:00  BTCUSDT  4333.32  4377.85  4333.32  4360.69   
53980  2017-08-17 07:00:00  BTCUSDT  4316.62  4349.99  4287.41  4349.99   
53981  2017-08-17 06:00:00  BTCUSDT  4330.29  4345.45  4309.37  4324.35   
53982  2017-08-17 05:00:00  BTCUSDT  4308.83  4328.69  4291.37  4315.32   
53983  2017-08-17 04:00:00  BTCUSDT  4261.48  4313.62  4261.32  4308.83   

       volume btc    volume usdt     volume        rsi  ...      tr     atr  \
53979    0.972807    4239.503586   0.972807  32.492544  ...  110.68  80.652   
53980    4.443249   19241.058300   4.443249  30.057888  ...   73.28  85.708   
53981    7.229691   31282.312670   7.229691  24.251939  ...   40.62  78.954   
53982   23.234916  100304.823567  23.234916  22.235138  ...   37.32  72.380   
53983   47.181009  202366.138393  47.181009  20.593972  ...   54.00  63.180   

               vwap        adx     gap  gap_percent  position_size  \
5397

FEATURE SCALING

In [60]:
#Features for HMM, LSTM & XGBoost
features = ['close', 'volume btc', 'rsi', 'macd', 'macd_signal', 'vwap', 'ema_20', 'ema_50', 'atr']
target = 'trade_signal'

#Drop rows with missing values
df.dropna(subset=features + [target], inplace=True)

df['trade_signal'] = df['trade_signal'].astype(int)

#Scale features
scaler_xgb = StandardScaler()
scaler_lstm = MinMaxScaler()

#Split dataset (60% train, 40% test)
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], 
                                                    test_size=0.4, train_size=0.6, random_state=42, stratify=df[target])

#Print data overview
print("Data loaded & preprocessed successfully.")
print("Training Samples:", X_train.shape[0])
print("Testing Samples:", X_test.shape[0])

df.tail()


Data loaded & preprocessed successfully.
Training Samples: 32387
Testing Samples: 21592


,date,symbol,open,high,low,close,volume btc,volume usdt,volume,rsi,...,tr,atr,vwap,adx,gap,gap_percent,position_size,trade_signal,take_profit,stop_loss
53979,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4239.503586,0.972807,32.492544,...,110.68,80.652,21924.915979,95.330857,-110.68,-2.490549,0.5,0,4586.5156,NaN
53980,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,19241.058300,4.443249,30.057888,...,73.28,85.708,21924.915512,95.200286,-44.07,-1.010620,0.5,0,4589.9724,NaN
53981,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,31282.312670,7.229691,24.251939,...,40.62,78.954,21924.914754,95.136857,-19.70,-0.452875,0.5,0,4545.4212,NaN
53982,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,100304.823567,23.234916,22.235138,...,37.32,72.380,21924.912314,94.116143,-15.52,-0.358898,0.5,0,4517.9840,NaN
53983,2017-08-17 04:00:00,BTCUSDT,4261.48,4313.62,4261.32,4308.83,47.181009,202366.138393,47.181009,20.593972,...,54.00,63.180,21924.907355,91.724143,-53.84,-1.247648,0.5,0,4485.7340,NaN


In [61]:
print(df.tail())

                      date   symbol     open     high      low    close  \
53979  2017-08-17 08:00:00  BTCUSDT  4333.32  4377.85  4333.32  4360.69   
53980  2017-08-17 07:00:00  BTCUSDT  4316.62  4349.99  4287.41  4349.99   
53981  2017-08-17 06:00:00  BTCUSDT  4330.29  4345.45  4309.37  4324.35   
53982  2017-08-17 05:00:00  BTCUSDT  4308.83  4328.69  4291.37  4315.32   
53983  2017-08-17 04:00:00  BTCUSDT  4261.48  4313.62  4261.32  4308.83   

       volume btc    volume usdt     volume        rsi  ...      tr     atr  \
53979    0.972807    4239.503586   0.972807  32.492544  ...  110.68  80.652   
53980    4.443249   19241.058300   4.443249  30.057888  ...   73.28  85.708   
53981    7.229691   31282.312670   7.229691  24.251939  ...   40.62  78.954   
53982   23.234916  100304.823567  23.234916  22.235138  ...   37.32  72.380   
53983   47.181009  202366.138393  47.181009  20.593972  ...   54.00  63.180   

               vwap        adx     gap  gap_percent  position_size  \
5397

HMM (MARKET REGIME CLASSIFICATION)

In [78]:
import joblib

# Train a Gaussian HMM with 3 states (bullish, bearish, sideways)
hmm_model = hmm.GaussianHMM(n_components=5, covariance_type="full", n_iter=150)
hmm_model.fit(X_train)

# Predict market regimes
market_regime_train = hmm_model.predict(X_train)
market_regime_test = hmm_model.predict(X_test)

# Add as a feature
X_train_hmm = np.column_stack((X_train, market_regime_train))
X_test_hmm = np.column_stack((X_test, market_regime_test))

joblib.dump(hmm_model, 'hmm_model.pkl')


['hmm_model.pkl']

LSTM (TREND PREDICTION)

In [79]:
import joblib

# Reshape data for LSTM (samples, timesteps, features)
X_train_lstm = X_train_hmm.reshape((X_train_hmm.shape[0], 1, X_train_hmm.shape[1]))
X_test_lstm = X_test_hmm.reshape((X_test_hmm.shape[0], 1, X_test_hmm.shape[1]))

# Build LSTM Model
lstm_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train_hmm.shape[1])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (Up or Down)
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train LSTM
lstm_model.fit(X_train_lstm, y_train, epochs=25, batch_size=64, verbose=1)

# Predict trend direction
lstm_pred_train = lstm_model.predict(X_train_lstm).flatten()
lstm_pred_test = lstm_model.predict(X_test_lstm).flatten()

# Convert LSTM output to binary (0 = Down, 1 = Up)
lstm_pred_train = (lstm_pred_train > 0.5).astype(int)
lstm_pred_test = (lstm_pred_test > 0.5).astype(int)

# Add as a feature
X_train_final = np.column_stack((X_train_hmm, lstm_pred_train))
X_test_final = np.column_stack((X_test_hmm, lstm_pred_test))

joblib.dump(lstm_model, 'lstm_model.h5')


Epoch 1/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9051 - loss: 0.4051
Epoch 2/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9232 - loss: 0.3579
Epoch 3/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9234 - loss: 0.3591
Epoch 4/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9233 - loss: 0.3578
Epoch 5/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9228 - loss: 0.3649
Epoch 6/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9228 - loss: 0.3641
Epoch 7/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9223 - loss: 0.3627
Epoch 8/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9219 - loss: 0.3611
Epoch 9/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9237 - loss: 0.3608
Epoch 10/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9215 - loss: 0.3685
Epoch 11/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9234 - loss: 0.3582
Epoch 12/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

['lstm_model.h5']

LSTM (FORECASTING PRICE)

In [80]:
import joblib

# Reshape data for LSTM (samples, timesteps, features)
X_train_lstm = X_train_hmm.reshape((X_train_hmm.shape[0], 1, X_train_hmm.shape[1]))
X_test_lstm = X_test_hmm.reshape((X_test_hmm.shape[0], 1, X_test_hmm.shape[1]))

# Extract indices from the original DataFrame
train_indices = df.index[:len(X_train_hmm)]  
test_indices = df.index[len(X_train_hmm):len(X_train_hmm) + len(X_test_hmm)]

# Define the target for price prediction (next closing price)
y_train_price = df.loc[train_indices, 'close'].shift(-1).dropna().values
# Align X_train_lstm with y_train_price
X_train_lstm = X_train_lstm[:len(y_train_price)]  # Trim X_train_lstm to match y_train_price length


y_test_price = df.loc[test_indices, 'close'].shift(-1).fillna(method='ffill').values


# --- LSTM Model for Price Prediction ---
price_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train_hmm.shape[1])),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(1, activation='linear')  # Predict price
])

price_model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# Train the price prediction LSTM
price_model.fit(X_train_lstm, y_train_price, epochs=25, batch_size=64, verbose=1)

# Predict next price
predicted_price_train = price_model.predict(X_train_lstm).flatten()
predicted_price_test = price_model.predict(X_test_lstm).flatten()

# Add price prediction as feature for XGBoost
X_train_final = np.column_stack((X_train_hmm[:len(predicted_price_train)], lstm_pred_train[:len(predicted_price_train)], predicted_price_train))
X_test_final = np.column_stack((X_test_hmm, lstm_pred_test, predicted_price_test))

joblib.dump(price_model, 'price_model.h5')

Epoch 1/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1069323840.0000 - mae: 28984.7168
Epoch 2/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1058221696.0000 - mae: 28769.9922
Epoch 3/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1063896768.0000 - mae: 28898.7559
Epoch 4/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1062689024.0000 - mae: 28836.7227
Epoch 5/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1061714880.0000 - mae: 28839.5059
Epoch 6/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1061041344.0000 - mae: 28783.6152
Epoch 7/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1064059776.0000 - mae: 28900.5312
Epoch 8/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1052587392.0000 - mae: 28688.0977
Epoch 9/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1057571456.0000 - mae: 28741.8750
Epoch 10/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1061679936.0000 - mae: 28835.5566
Epoch 11/25
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss

['price_model.h5']

XGBOOST (GENERATE SIGNALS BASED ON HMM AND LSTM)

In [65]:
import joblib

predicted_price_train = predicted_price_train[:X_train.shape[0]]
predicted_price_test = predicted_price_test[:X_test.shape[0]]
X_train = X_train[:predicted_price_train.shape[0]]
market_regime_train = market_regime_train[:predicted_price_train.shape[0]]
lstm_pred_train = lstm_pred_train[:predicted_price_train.shape[0]]

X_train_final = X_train_final[:y_train.shape[0]]  # Trim features to match labels
y_train = y_train[:X_train_final.shape[0]]  # Or trim labels to match features

# Append HMM & both LSTM outputs (Trend & Price) to feature set
X_train_final = np.column_stack((X_train, market_regime_train, lstm_pred_train, predicted_price_train))
X_test_final = np.column_stack((X_test, market_regime_test, lstm_pred_test, predicted_price_test))

# Train XGBoost Model
model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    eval_metric='mlogloss',
    n_estimators=350,
    max_depth=5,
    learning_rate=0.03,
    subsample=0.9,
    colsample_bytree=0.9
)

model.fit(X_train_final, y_train)

# Make Predictions
y_pred = model.predict(X_test_final)

results_df = pd.DataFrame({
    "market_regime": market_regime_test,
    "predicted_price": predicted_price_test,
    "model_signal": y_pred
}, index=range(len(df) - len(X_test), len(df)))  # Create matching index

df.update(results_df)  # Merge results into `df`


# Performance Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Check Prediction Distribution
print(pd.Series(y_pred).value_counts())

df.tail()

joblib.dump(model, 'xgb_model.pkl')


Accuracy: 0.9246943312337903
F1 Score: 0.8994660711440639
0    21261
2      195
1      136
Name: count, dtype: int64


['xgb_model.pkl']

TRADE PERFORMANCE METRICS

In [66]:
#Trade Frequency (Percentage of Time Market is Active)
trade_count = df['trade_signal'].abs().sum()
trade_frequency = (trade_count / len(df)) * 100

# 🔹 ATR Calculation (Using Proper True Range Formula)
df['tr'] = np.maximum.reduce([
    df['high'] - df['low'],
    abs(df['high'] - df['close'].shift(1)),
    abs(df['low'] - df['close'].shift(1))
])
df['atr'] = df['tr'].ewm(span=14, adjust=False).mean()

#Stop-Loss & Take-Profit Levels (Adjusted for Market Volatility)
df['stop_loss'] = df['close'] - (df['atr'] * 1.0)
df['take_profit'] = df['close'] + (df['atr'] * 4.0) 

#Dynamic Position Sizing (Risking 2% of Account Per Trade)
account_risk = 0.02  # 2% Risk per trade
df['position_size'] = (account_risk / (df['atr'] + 1e-6)).clip(0.01, 0.1)  # Prevent extreme values

#Strategy Returns Calculation (Includes Position Sizing)
df['strategy_returns'] = df['trade_signal'].shift(1) * (df['close'].pct_change() * df['position_size'])

#Cumulative Returns Calculation
df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

#Max Drawdown Calculation (Using Peak-to-Trough Decline)
df['rolling_max'] = df['cumulative_returns'].cummax()
df['drawdown'] = (df['cumulative_returns'] - df['rolling_max']) / df['rolling_max']
max_drawdown = df['drawdown'].min()

#Filter Only Buy and Sell for Sharpe Ratio Calculation
trade_returns = df[df['trade_signal'] != 0]['strategy_returns']

#Sharpe Ratio Calculation (Annualized)
if not trade_returns.empty:
    mean_strategy_returns = trade_returns.mean()
    std_strategy_returns = trade_returns.std()
    sharpe_ratio = (mean_strategy_returns / (std_strategy_returns + 1e-6)) 
else:
    sharpe_ratio = 0 #To Avoid Division By Zero (If No Trades)

#Profit Factor Calculation (Total Gains vs. Total Losses)
total_gains = trade_returns[trade_returns > 0].sum()
total_losses = abs(trade_returns[trade_returns < 0].sum())
profit_factor = total_gains / total_losses if total_losses > 0 else float('inf')

#Accuracy Calculation (Only for ML Predictions)
if 'y_test' in locals() and 'y_pred' in locals():
    from sklearn.metrics import accuracy_score
    # Filter out "0" (Hold) predictions
    trade_indices = (y_pred != 0)
    trade_predictions = y_pred[trade_indices]
    trade_actuals = y_test[trade_indices]

    # Compute accuracy only on Buy (1) and Sell (2)
    if len(trade_actuals) > 0:
        trade_accuracy = accuracy_score(trade_actuals, trade_predictions) * 100
    else:
        trade_accuracy = 0  # Avoid division by zero

    print(f"Trade Accuracy (Buy/Sell Only): {trade_accuracy:.2f}%")

#Performance Summary
print(f"Profit Factor: {profit_factor:.2f}")
print(f"Trade Frequency: {trade_frequency:.2f}%")
print(f"Estimated Max Drawdown: {max_drawdown:.2%}")
print(f"Estimated Sharpe Ratio: {sharpe_ratio:.2f}")





Trade Accuracy (Buy/Sell Only): 43.50%
Profit Factor: 0.49
Trade Frequency: 11.82%
Estimated Max Drawdown: -25.08%
Estimated Sharpe Ratio: -0.12


In [7]:
import joblib
from tensorflow.keras.models import load_model

class TradingModel:
    def __init__(self, hmm_model, lstm_model, price_model, model):
        self.hmm_model = hmm_model
        self.lstm_model = lstm_model
        self.price_model = price_model
        self.model = model

    def extract_hmm_features(self, X):
        return self.hmm_model.predict(X)
    
    def extract_lstm_trend(self, X):
        X_lstm = X.reshape((X.shape[0], 1, X.shape[1]))  # Reshape for LSTM
        lstm_pred = self.lstm_model.predict(X_lstm).flatten()
        return (lstm_pred > 0.5).astype(int)  # Convert to binary (0 or 1)
    
    def extract_lstm_price(self, X):
        X_lstm = X.reshape((X.shape[0], 1, X.shape[1]))  # Reshape for LSTM
        return self.price_model.predict(X_lstm).flatten()

    def predict(self, X):
        market_regime_test = self.extract_hmm_features(X)
        lstm_pred_test = self.extract_lstm_trend(X)
        predicted_price_test = self.extract_lstm_price(X)

        X_test_final = np.column_stack([X, market_regime_test, lstm_pred_test, predicted_price_test])
        return self.model.predict(X_test_final)

# Load trained models
hmm_model = joblib.load('hmm_model.pkl')
price_model = load_model('price_model.h5')
model = joblib.load('xgb_model.pkl')

# Create and save full model
full_model = TradingModel(hmm_model, lstm_model, price_model, model)
joblib.dump(full_model, 'trading_model.pkl')

# Example Usage
loaded_model = joblib.load('trading_model.pkl')
y_pred = loaded_model.predict(X_test)
print("Predictions:", y_pred)

OSError: Unable to synchronously open file (file signature not found)